# Housing Price Prediction — ETL

**Project:** Housing Price Prediction Dashboard  

**Notebook:** ETL (Extract, Transform, Load) Data

---

## Overview
This notebook performs the ETL process for the Housing Price dataset.  
Tasks include:
- Importing raw data  
- Cleaning missing and inconsistent values  
- Feature engineering  
- Exploratory data analysis (EDA)  
- Saving processed data for model training and dashboard use  

---


## 1) Import Libraries

In [1]:
import pandas as pd
import numpy as np
import re

## 2) ETL Pipeline

### Extract

In [2]:
# EXTRACT
def extract(raw_path: str):
    try:
        df = pd.read_csv(raw_path)
        print(f"Loaded dataset successfully from: {raw_path}")
        display(df.head())
        return df
    except FileNotFoundError:
        print(f"Error: File not found → {raw_path}")
    except Exception as e:
        print(f"Unexpected error: {e}")

### Transform

In [3]:
# TRANSFORM
def transform(df):
    """
    Transform step of ETL pipeline.
    Cleans dataframe, handles missing values,
    formats columns, and engineers features.
    """

    df = df.copy()
    print("Starting TRANSFORM step...")
    
    # Standardise column names
    df.columns = (
        df.columns
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
    )
    print("Column names standardized.")

    # Remove duplicates
    before_dup = df.shape[0]
    df = df.drop_duplicates()
    after_dup = df.shape[0]
    print(f"Removed duplicates: {before_dup - after_dup} rows dropped.")

    # Handlings for missing values
    print("Checking missing values:")
    null_counts = df.isnull().sum()
    print(null_counts[null_counts > 0] if null_counts.sum() > 0 else "No missing values found.")

    # Handle missing values (median imputation for numeric columns and mode imputation for categorical columns)
    num_cols = df.select_dtypes(include=["number"]).columns
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())
    print(f"Filled missing values in numeric columns: {list(num_cols)}")

    cat_cols = df.select_dtypes(include=["object"]).columns
    df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])
    print(f"Filled missing values in categorical columns: {list(cat_cols)}")

    # Fix naming conventions (upper to capitalise)
    string_cols = ['town', 'flat_type', 'street_name', 'storey_range']
    for col in string_cols:
        df[col] = df[col].str.title()
    print(f"Capitalized string columns: {string_cols}")

    # Feature engineering
    df['year'] = pd.to_datetime(df['month'], format='%Y-%m').dt.year
    df['month'] = pd.to_datetime(df['month'], format='%Y-%m').dt.strftime('%b')
    print("Extracted 'year' and 'month_num' from 'month' column.")

    if 'remaining_lease' in df.columns:
        def lease_to_years(lease_str):
            if pd.isnull(lease_str):
                return None

            lease_str = lease_str.lower().strip()

            # Extract years
            years_match = re.search(r'(\d+)\s*years?', lease_str)
            years = int(years_match.group(1)) if years_match else 0

            # Extract months (optional)
            months_match = re.search(r'(\d+)\s*months?', lease_str)
            months = int(months_match.group(1)) if months_match else 0

            total_years = years + months / 12
            return total_years if total_years > 0 else None

        df.loc[:, 'remaining_lease_years'] = df['remaining_lease'].apply(lease_to_years)
        print("Converted 'remaining_lease' to numeric years in 'remaining_lease_years'.")

    # Reorder columns
    df = df[sorted(df.columns)]


    print("Transform step completed. Dataset shape:", df.shape)
    print("Dataset head:")
    display(df.head())
    return df

### Load

In [4]:
# LOAD
def load(df, output_path="data/cleaned_resale_flat.csv"):
    """
    Load step of ETL pipeline.
    Saves the cleaned dataframe to disk.
    """
    df.to_csv(output_path, index=False)
    print(f"Data loaded successfully to {output_path}.")

### Full ETL

In [5]:
def ETL(input, output):
    """
    Runs the full ETL pipeline: extract, transform, load.
    """
    # Extract
    data = extract(input)
    
    # Transform
    data = transform(data)
    
    # Load
    load(data, output)
    
    print("ETL pipeline completed successfully.")
    return data

## 3) Save cleaned dataset

In [6]:
input = 'data/resale_flat_price.csv'
output = 'data/cleaned_resale_flat_price.csv'
ETL(input, output)

Loaded dataset successfully from: data/resale_flat_price.csv


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


Starting TRANSFORM step...
Column names standardized.
Removed duplicates: 306 rows dropped.
Checking missing values:
No missing values found.
Filled missing values in numeric columns: ['floor_area_sqm', 'lease_commence_date', 'resale_price']
Filled missing values in categorical columns: ['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'flat_model', 'remaining_lease']
Capitalized string columns: ['town', 'flat_type', 'street_name', 'storey_range']
Extracted 'year' and 'month_num' from 'month' column.
Converted 'remaining_lease' to numeric years in 'remaining_lease_years'.
Transform step completed. Dataset shape: (218304, 13)
Dataset head:


,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,remaining_lease_years,resale_price,storey_range,street_name,town,year
0,406,Improved,2 Room,44.0,1979,Jan,61 years 04 months,61.333333,232000.0,10 To 12,Ang Mo Kio Ave 10,Ang Mo Kio,2017
1,108,New Generation,3 Room,67.0,1978,Jan,60 years 07 months,60.583333,250000.0,01 To 03,Ang Mo Kio Ave 4,Ang Mo Kio,2017
2,602,New Generation,3 Room,67.0,1980,Jan,62 years 05 months,62.416667,262000.0,01 To 03,Ang Mo Kio Ave 5,Ang Mo Kio,2017
3,465,New Generation,3 Room,68.0,1980,Jan,62 years 01 month,62.083333,265000.0,04 To 06,Ang Mo Kio Ave 10,Ang Mo Kio,2017
4,601,New Generation,3 Room,67.0,1980,Jan,62 years 05 months,62.416667,265000.0,01 To 03,Ang Mo Kio Ave 5,Ang Mo Kio,2017


Data loaded successfully to data/cleaned_resale_flat_price.csv.
ETL pipeline completed successfully.


,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,remaining_lease_years,resale_price,storey_range,street_name,town,year
0,406,Improved,2 Room,44.0,1979,Jan,61 years 04 months,61.333333,232000.0,10 To 12,Ang Mo Kio Ave 10,Ang Mo Kio,2017
1,108,New Generation,3 Room,67.0,1978,Jan,60 years 07 months,60.583333,250000.0,01 To 03,Ang Mo Kio Ave 4,Ang Mo Kio,2017
2,602,New Generation,3 Room,67.0,1980,Jan,62 years 05 months,62.416667,262000.0,01 To 03,Ang Mo Kio Ave 5,Ang Mo Kio,2017
3,465,New Generation,3 Room,68.0,1980,Jan,62 years 01 month,62.083333,265000.0,04 To 06,Ang Mo Kio Ave 10,Ang Mo Kio,2017
4,601,New Generation,3 Room,67.0,1980,Jan,62 years 05 months,62.416667,265000.0,01 To 03,Ang Mo Kio Ave 5,Ang Mo Kio,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...
218605,877,Maisonette,Executive,145.0,1987,Sep,61 years 03 months,61.250000,980000.0,10 To 12,Yishun St 81,Yishun,2025
218606,834,Maisonette,Executive,146.0,1988,Sep,61 years 04 months,61.333333,990000.0,04 To 06,Yishun St 81,Yishun,2025
218607,834,Apartment,Executive,142.0,1988,Oct,61 years 04 months,61.333333,990000.0,07 To 09,Yishun St 81,Yishun,2025
218608,632,Multi Generation,Multi-Generation,147.0,1987,May,61 years 06 months,61.500000,945000.0,04 To 06,Yishun St 61,Yishun,2025
